In [1]:
import yaml
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from phase_1_get_dates import *

# PHASE 1

# Step 1: Fetch historical close prices
tickers = ['SPY']
end_date = datetime(2025, 6, 1)
start_date = end_date - timedelta(days=365 * 10) # datetime(2025, 6, 1)
historic_close_data = fetch_historical_close_prices(tickers, start_date, end_date)

# Step 2: Calculate technical indicators for multiple configurations
configs_technical_indicators = {'config_ti1': {'window': 20, 'std_dev': 1.0}, 'config_ti2': {'window': 20, 'std_dev': 2.0}}
dict_technical_indicators = generate_dict_technical_indicators(
    historic_close_data, 
    configs_technical_indicators, 
    print_sample=False
)

# Step 3: Filter daily tickers by technical indicators
configs_technical_indicator_filter = {
    'config_tif1': lambda df: df['close'] <= df['bb_upper'],
    'config_tif2': lambda df: df['close'] <= df['bb_lower'],
    'config_tif3': lambda df: (df['close'].shift(-1) <= df['bb_lower'].shift(-1)) & (df['close'] >= df['bb_lower']),
}

dict_technical_indicator_filter = generate_dict_technical_indicator_filter(
    dict_technical_indicators, 
    historic_close_data,
    configs_technical_indicator_filter
)

# Step 4: Extract entry dates
entry_dates_dict = extract_entry_dates(dict_technical_indicator_filter)
entry_dates_dict_str = extract_entry_dates(dict_technical_indicator_filter, as_string=True)



Fetching data for ['SPY'] from 2015-06-04 to 2025-06-01...


[*********************100%***********************]  1 of 1 completed

config_ti1 | config_tif1 | SPY: 1382 entries pass filter (out of 2494 total)
config_ti1 | config_tif2 | SPY: 427 entries pass filter (out of 2494 total)
config_ti1 | config_tif3 | SPY: 117 entries pass filter (out of 2494 total)
config_ti2 | config_tif1 | SPY: 2345 entries pass filter (out of 2494 total)
config_ti2 | config_tif2 | SPY: 126 entries pass filter (out of 2494 total)
config_ti2 | config_tif3 | SPY: 69 entries pass filter (out of 2494 total)


In [ ]:
# ? for config in configs...


In [ ]:
# Step 1: Select date range within above

# Step 2: Get options chain data

# Step 3: Filter options chain data by DTE, Delta, and IV

# # # # Technical Filter Configurations with associated list of tickers, start date, end date, and parameters
# # # # Example:
# # # # technical_filter_config = {
# # # #     'name': 'SMA_20_BB_UPPER',
# # # #     'tickers': ['TSLA'],
# # # #     'start_date': '2025-01-01',
# # # #     'end_date': '2025-12-31',

In [3]:
import os

In [5]:
entry_dates_dict_str.keys()

dict_keys(['config_ti1', 'config_ti2'])

In [ ]:
# Create a set of the dates in entry_dates_dict_str
# Need a config map for technical configs to put configs
# Need to make sure still that we are pull a date across all configs once
# Get that row in options data, generate the additional stats. 
# Add them to portfolio



In [ ]:
# =============================================================================
# PUT CONFIGURATION (strategy-specific)
# =============================================================================

# Base PUT config template
PUT_CONFIG_BASE = {
    # -------------------------------------------------------------------------
    # SYMBOL & TIMING
    # -------------------------------------------------------------------------
    'entry_date_start': '2023-06-06',          # First Date to enter positions
    'entry_date_end': '2023-06-06',            # Last Date to enter positions
    'entry_time': '15:45',                     # Time to capture option chain snapshot
    
    # -------------------------------------------------------------------------
    # OPTION SELECTION CRITERIA
    # -------------------------------------------------------------------------
    'option_type': 'P',                        # 'P' for puts (CSP), 'C' for calls
    'dte_min': None,                             # Minimum days to expiration
    'dte_max': None,                             # Maximum days to expiration
    'delta_min': None,                         # Minimum absolute delta
    'delta_max': None,                         # Maximum absolute delta
    
    # -------------------------------------------------------------------------
    # EXIT STRATEGY
    # -------------------------------------------------------------------------
    'exit_pct': None,                          # 0.50 = buy back at 50%, keep 50% profit
    'stop_loss_multiplier': None,               # Exit if option price reaches Nx premium
    'max_hold_dte': None,                      # Exit at X DTE if no other trigger (None = disabled)
}

# =============================================================================
# MULTIPLE PUT CONFIGS FOR TESTING
# =============================================================================

# Conservative strategy: Longer DTE, lower delta, tighter exit
PUT_CONFIG_CONSERVATIVE = {
    **PUT_CONFIG_BASE,
    'dte_min': 45,
    'dte_max': 60,
    'delta_min': 0.20,
    'delta_max': 0.30,
    'exit_pct': 0.30,                         # Exit at 30% profit (more conservative)
    'stop_loss_multiplier': 1.5,              # Tighter stop loss
    'name': 'conservative',                   # Identifier for this config
}

# Aggressive strategy: Shorter DTE, higher delta, wider exit
PUT_CONFIG_AGGRESSIVE = {
    **PUT_CONFIG_BASE,
    'dte_min': 21,
    'dte_max': 35,
    'delta_min': 0.30,
    'delta_max': 0.40,
    'exit_pct': 0.70,                          # Exit at 70% profit (let it run)
    'stop_loss_multiplier': 3.0,               # Wider stop loss
    'name': 'aggressive',
}

# Balanced strategy (default)
PUT_CONFIG_BALANCED = {
    **PUT_CONFIG_BASE,
    'name': 'balanced',
}

# Quick profit strategy: Very tight exit, moderate delta
PUT_CONFIG_QUICK_PROFIT = {
    **PUT_CONFIG_BASE,
    'dte_min': 30,
    'dte_max': 45,
    'delta_min': 0.25,
    'delta_max': 0.35,
    'exit_pct': 0.25,                          # Exit at 25% profit (quick exits)
    'stop_loss_multiplier': 2.0,
    'max_hold_dte': 7,                         # Force exit at 7 DTE
    'name': 'quick_profit',
}

# High premium strategy: Higher delta for more premium
PUT_CONFIG_HIGH_PREMIUM = {
    **PUT_CONFIG_BASE,
    'dte_min': 30,
    'dte_max': 45,
    'delta_min': 0.35,
    'delta_max': 0.45,
    'exit_pct': 0.50,
    'stop_loss_multiplier': 2.5,
    'name': 'high_premium',
}

# =============================================================================
# HELPER FUNCTION: Merge shared and put configs
# =============================================================================
def create_full_config(put_config):
    """
    Merge SHARED_CONFIG with a PUT_CONFIG to create a complete config.
    
    Args:
        put_config: One of the PUT_CONFIG dictionaries
    
    Returns:
        Complete config dictionary with all settings
    """
    # Merge shared config with put config (put_config takes precedence for any overlaps)
    full_config = {**SHARED_CONFIG, **put_config}
    return full_config

# =============================================================================
# USAGE EXAMPLE
# =============================================================================

# Test with balanced config
CONFIG = create_full_config(PUT_CONFIG_BALANCED)

# Or test with aggressive config
# CONFIG = create_full_config(PUT_CONFIG_AGGRESSIVE)

# Or test with conservative config
# CONFIG = create_full_config(PUT_CONFIG_CONSERVATIVE)

# =============================================================================
# DERIVED VALUES (computed from CONFIG)
# =============================================================================
TZ = CONFIG['timezone']
CACHE_DIR = CONFIG['cache_dir']
os.makedirs(CACHE_DIR, exist_ok=True)

# Entry timestamp
ENTRY_DATE_START = pd.Timestamp(CONFIG['entry_date_start'], tz=TZ)
ENTRY_DATE_END = pd.Timestamp(CONFIG['entry_date_end'], tz=TZ)
ENTRY_TIME = pd.Timestamp(f"{CONFIG['entry_date_start']} {CONFIG['entry_time']}", tz=TZ)

print("=" * 60)
print("BACKTEST CONFIGURATION")
print("=" * 60)
print(f"Config Name:     {CONFIG.get('name', 'default')}")
print(f"Entry Date Start: {ENTRY_DATE_START.date()}")
print(f"Entry Date End:   {ENTRY_DATE_END.date()}")
print(f"Entry Time:       {CONFIG['entry_time']}")
print(f"Option Type:      {'Cash-Secured Put' if CONFIG['option_type'] == 'P' else 'Covered Call'}")
print(f"DTE Range:        {CONFIG['dte_min']} - {CONFIG['dte_max']} days")
print(f"Delta Range:      {CONFIG['delta_min']} - {CONFIG['delta_max']}")
print(f"Exit Target:      {CONFIG['exit_pct']*100:.0f}% of premium")
print(f"Stop Loss:        {CONFIG['stop_loss_multiplier']}x premium")
print(f"Fill Mode:        {CONFIG['fill_mode']}")
print(f"Commission:       ${CONFIG['commission_per_contract']}/contract")
print("=" * 60)


BACKTEST CONFIGURATION
Config Name:     balanced
Entry Date Start: 2023-06-06
Entry Date End:   2023-06-06
Entry Time:       15:45
Option Type:      Cash-Secured Put
DTE Range:        30 - 45 days
Delta Range:      0.25 - 0.35
Exit Target:      50% of premium
Stop Loss:        2.0x premium
Fill Mode:        mid
Commission:       $0.65/contract


In [ ]:
# I think this can come at the end in backtest calculations so long as we calculate and store the variables

# =============================================================================
# SHARED CONFIGURATION (used across all strategies)
# =============================================================================
SHARED_CONFIG = {
    # -------------------------------------------------------------------------
    # LIQUIDITY MODEL (regime-aware, penalty-based)
    # -------------------------------------------------------------------------
    # Hard rejection thresholds (truly untradeable)
    'min_bid_hard': 0.10,                      # Hard floor - reject penny options
    'hard_max_spread_pct': 0.20,               # Hard ceiling - reject extreme spreads
    
    # Base target spread (calm market conditions)
    'base_max_spread_pct': 0.08,               # Target max spread in normal conditions
    
    # IV regime adjustments (allow wider spreads in high-vol)
    'ivp_high_threshold': 0.70,                # IV percentile threshold for "high vol"
    'ivp_high_max_spread_pct': 0.12,           # Allowed spread when IV is high
    'ivp_extreme_threshold': 0.90,             # IV percentile threshold for "extreme vol"
    'ivp_extreme_max_spread_pct': 0.15,        # Allowed spread when IV is extreme
    
    # DTE adjustments (short-dated options have wider spreads)
    'short_dte_threshold': 7,                  # DTE below this gets extra allowance
    'short_dte_extra_spread_pct': 0.02,        # Extra spread allowance for short DTE
    
    # Penalty tiers (execution tax based on spread quality)
    # tight:    spread <= 0.6 * allowed → penalty = 1.0 (no extra slippage)
    # moderate: spread <= allowed       → penalty = 1.15 (15% wider effective spread)
    # wide:     spread <= hard_max      → penalty = 1.35 (35% wider effective spread)
    # ugly:     spread > hard_max       → REJECT (no trade)
    
    # -------------------------------------------------------------------------
    # TRANSACTION COSTS
    # -------------------------------------------------------------------------
    'commission_per_contract': 0.65,           # Per contract commission (round trip = 2x)
    'sec_fee_per_contract': 0.01,              # SEC/TAF fees per contract
    
    # -------------------------------------------------------------------------
    # EXECUTION / FILL ASSUMPTIONS
    # -------------------------------------------------------------------------
    'fill_mode': 'mid',                        # 'mid' (current), 'bid' (realistic), 'pessimistic'
    'use_realistic_fills': False,              # When True: sell at bid, buy back at ask
    
    # -------------------------------------------------------------------------
    # PROBABILISTIC EXIT FILLS
    # -------------------------------------------------------------------------
    'execution_seed': 42,                      # Random seed for reproducible fills
    'use_probabilistic_exit_fills': True,      # Enable probabilistic fill model
    
    # Fill probability buckets by spread quality
    'pfill_tight': 0.90,                       # spread <= 5%
    'pfill_normal': 0.70,                      # spread <= 10%
    'pfill_wide': 0.40,                        # spread > 10%
    
    # Spread thresholds for buckets
    'tight_spread_pct': 0.05,
    'normal_spread_pct': 0.10,
    
    # Scaling and clamping
    'pfill_scale': 0.8,                        # Sensitivity multiplier (0.8, 1.0, 1.2)
    'pfill_min': 0.05,
    'pfill_max': 0.98,
    
    # Optional IVP penalty multipliers
    'pfill_ivp_high_mult': 0.85,
    'pfill_ivp_extreme_mult': 0.70,
    
    # -------------------------------------------------------------------------
    # CACHE
    # -------------------------------------------------------------------------
    'cache_dir': '../cache/',
    
    # -------------------------------------------------------------------------
    # TIMEZONE
    # -------------------------------------------------------------------------
    'timezone': 'America/New_York',
}


In [12]:
# =============================================================================
# HELPER FUNCTIONS FOR REALISTIC EXECUTION
# =============================================================================

def get_entry_price(row, fill_mode='realistic', penalty=1.0):
    """
    Calculate entry price when SELLING a put (we receive premium).
    Higher price = better for us.
    
    Slippage is calculated as a percentage of the bid-ask spread from mid.
    Penalty multiplier widens the effective spread for illiquid options.
    
    | Scenario    | Formula                              | Interpretation              |
    |-------------|--------------------------------------|-----------------------------|
    | pessimistic | mid - 75% of (spread * penalty)      | Forced/stressed execution   |
    | realistic   | mid - 30% of (spread * penalty)      | Normal retail execution     |
    | optimistic  | mid                                  | Patient, favorable fills    |
    
    Args:
        row: DataFrame row with bid_px_00, ask_px_00
        fill_mode: 'optimistic', 'realistic', or 'pessimistic'
        penalty: liquidity penalty multiplier (1.0 = no extra slippage)
    """
    
    bid = row['bid_px_00']
    ask = row['ask_px_00']
    mid = (bid + ask) / 2
    spread = ask - bid
    
    # Apply liquidity penalty to effective spread
    effective_spread = spread * penalty
    
    if fill_mode == 'optimistic':
        return mid                              # Best case - get mid (no penalty applied)
    elif fill_mode == 'pessimistic':
        fill = mid - (0.75 * effective_spread)  # Worst case - 75% toward bid
    else:  # realistic
        fill = mid - (0.30 * effective_spread)  # Normal - 30% toward bid
    
    # Clamp to [bid, ask] to stay realistic
    return max(bid, min(ask, fill))

In [13]:
def get_exit_price(daily_row, fill_mode=CONFIG['fill_mode'], target_price=None, penalty=1.0):
    """
    Calculate exit price when BUYING BACK a put (we pay to close).
    Lower price = better for us.
    
    For daily OHLCV data, we estimate spread behavior from the day's range.
    Penalty multiplier widens the effective range for illiquid options.
    
    | Scenario    | Formula                              | Interpretation              |
    |-------------|--------------------------------------|-----------------------------|
    | pessimistic | close + 75% of (range * penalty)     | Forced/stressed execution   |
    | realistic   | close + 30% of (range * penalty)     | Normal retail execution     |
    | optimistic  | close - 25% of (range * penalty)     | Patient, favorable fills    |
    
    Args:
        daily_row: DataFrame row with close, high, low
        fill_mode: 'optimistic', 'realistic', or 'pessimistic'
        target_price: Optional target price (not currently used but reserved)
        penalty: liquidity penalty multiplier (1.0 = no extra slippage)
    """
    close = daily_row['close']
    high = daily_row['high']
    low = daily_row['low']
    day_range = high - low  # Proxy for intraday spread/volatility
    
    # Apply liquidity penalty to effective range
    effective_range = day_range * penalty
    
    if fill_mode == 'optimistic':
        # Patient buyer - gets below close (toward low)
        fill = close - (0.25 * effective_range)
        return max(low, fill)
    elif fill_mode == 'pessimistic':
        # Forced buyer - pays above close (toward high)
        fill = close + (0.75 * effective_range)
        return min(high, fill)
    else:  # realistic
        # Normal execution - slight slippage above close
        fill = close + (0.30 * effective_range)
        return min(high, fill)


In [14]:
def get_transaction_costs(config, is_round_trip=True):
    """
    Calculate total transaction costs per contract.
    
    Args:
        config: CONFIG dict with commission and fee rates
        is_round_trip: True if both entry and exit, False if entry only (e.g., expired worthless)
    
    Returns:
        Total fees in dollars per contract
    """
    per_leg = config['commission_per_contract'] + config['sec_fee_per_contract']
    return per_leg * 2 if is_round_trip else per_leg

In [15]:
def compute_allowed_spread(row, config):
    """
    Compute the allowed spread percentage for a single option based on regime.
    
    Regime factors:
    - IV percentile (high vol → allow wider spreads)
    - DTE (short-dated → allow wider spreads)
    
    Returns: allowed_spread_pct for this option
    """
    base = config['base_max_spread_pct']
    
    # IV regime adjustment
    ivp = row.get('ivp', 0.5)  # Default to median if not computed
    if ivp >= config['ivp_extreme_threshold']:
        base = config['ivp_extreme_max_spread_pct']
    elif ivp >= config['ivp_high_threshold']:
        base = config['ivp_high_max_spread_pct']
    
    # DTE adjustment
    dte = row.get('dte', 30)
    if dte <= config['short_dte_threshold']:
        base += config['short_dte_extra_spread_pct']
    
    return base
    

In [16]:
def compute_liquidity_penalty(spread_pct, allowed_spread_pct, hard_max_spread_pct):
    """
    Compute liquidity penalty multiplier based on spread quality.
    
    Tiers:
    - tight:    spread <= 0.6 * allowed → penalty = 1.0 (no extra slippage)
    - moderate: spread <= allowed       → penalty = 1.15
    - wide:     spread <= hard_max      → penalty = 1.35
    - ugly:     spread > hard_max       → None (reject)
    
    Returns: (tier_name, penalty_multiplier) or (None, None) if rejected
    """
    if spread_pct > hard_max_spread_pct:
        return 'reject', None
    
    tight_threshold = 0.6 * allowed_spread_pct
    
    if spread_pct <= tight_threshold:
        return 'tight', 1.0
    elif spread_pct <= allowed_spread_pct:
        return 'moderate', 1.15
    else:  # spread_pct <= hard_max_spread_pct
        return 'wide', 1.35
        

In [17]:
def apply_liquidity_model(df, config):
    """
    Apply regime-aware liquidity model with penalty tiers.
    
    Instead of binary reject, this:
    1. Computes IV percentile (ivp) for regime detection
    2. Computes allowed_spread_pct per option (regime-aware)
    3. Assigns liquidity_tier and liquidity_penalty
    4. Only hard-rejects truly ugly spreads
    
    Args:
        df: DataFrame with option quotes (needs bid_px_00, ask_px_00, spread_pct, iv, dte)
        config: CONFIG dict with liquidity model settings
    
    Returns:
        DataFrame with liquidity columns added, ugly spreads removed
    """
    if len(df) == 0:
        return df
    
    df = df.copy()
    original_count = len(df)
    
    # Ensure required columns exist
    if 'spread_pct' not in df.columns:
        df['spread'] = df['ask_px_00'] - df['bid_px_00']
        df['spread_pct'] = df['spread'] / df['mid']
    
    # Step 1: Compute IV percentile (cross-sectional within this snapshot)
    if 'iv' in df.columns:
        df['ivp'] = df['iv'].rank(pct=True)
    else:
        df['ivp'] = 0.5  # Default to median if IV not available
    
    # Step 2: Compute allowed spread per option
    df['allowed_spread_pct'] = df.apply(
        lambda row: compute_allowed_spread(row, config), axis=1
    )
    
    # Step 3: Compute liquidity tier and penalty
    def get_tier_and_penalty(row):
        return compute_liquidity_penalty(
            row['spread_pct'], 
            row['allowed_spread_pct'],
            config['hard_max_spread_pct']
        )
    
    tiers_penalties = df.apply(get_tier_and_penalty, axis=1)
    df['liquidity_tier'] = tiers_penalties.apply(lambda x: x[0])
    df['liquidity_penalty'] = tiers_penalties.apply(lambda x: x[1])
    
    # Step 4: Hard reject only truly ugly spreads and penny options
    df = df[
        (df['liquidity_tier'] != 'reject') &
        (df['bid_px_00'] >= config['min_bid_hard'])
    ].copy()
    
    rejected = original_count - len(df)
    
    # Print diagnostics
    print(f"\n  Liquidity Model Applied:")
    print(f"    Original: {original_count} options")
    print(f"    Hard rejected: {rejected} ({rejected/original_count*100:.1f}%)")
    print(f"    Remaining: {len(df)} options")
    
    if len(df) > 0:
        tier_counts = df['liquidity_tier'].value_counts()
        print(f"    Tier breakdown: {dict(tier_counts)}")
        print(f"    Avg spread: {df['spread_pct'].mean()*100:.1f}%, Avg allowed: {df['allowed_spread_pct'].mean()*100:.1f}%")
        print(f"    Avg penalty: {df['liquidity_penalty'].mean():.2f}x")
    return df


In [18]:
def calculate_pnl(premium_received, exit_price_paid, fees, cost_basis):
    """
    Calculate P&L metrics for a trade.
    
    Args:
        premium_received: Premium collected when selling (contract value)
        exit_price_paid: Price paid to close position (contract value), 0 if expired worthless
        fees: Total transaction costs
        cost_basis: Capital at risk (strike * 100 for CSP)
    
    Returns:
        dict with pnl, pnl_pct, roc
    """
    pnl = premium_received - exit_price_paid - fees
    pnl_pct = (pnl / premium_received) * 100 if premium_received > 0 else 0
    roc = (pnl / cost_basis) * 100 if cost_basis > 0 else 0
    
    return {
        'pnl': pnl,
        'pnl_pct': pnl_pct,
        'roc': roc,
        'fees': fees
    }

In [19]:
def compute_p_fill_profit(row, config):
    """
    Compute probability of fill for profit target exit based on entry liquidity.
    
    Uses entry-time spread and IVP to determine fill probability:
    - tight spread (<=5%): high fill probability
    - normal spread (<=10%): moderate fill probability
    - wide spread (>10%): low fill probability
    
    Applies IVP penalty multipliers for high-volatility regimes.
    
    Args:
        row: DataFrame row with spread_pct_entry, ivp_entry
        config: CONFIG dict with fill probability settings
    
    Returns:
        float: Fill probability in [pfill_min, pfill_max]
    """
    spread_pct = row.get('spread_pct_entry', 0.05)
    
    # Bucket by spread quality
    if spread_pct <= config['tight_spread_pct']:
        p_fill = config['pfill_tight']
    elif spread_pct <= config['normal_spread_pct']:
        p_fill = config['pfill_normal']
    else:
        p_fill = config['pfill_wide']
    
    # Apply scale multiplier
    p_fill *= config['pfill_scale']
    
    # Apply IVP penalty (always use ivp_entry, not generic ivp)
    ivp = row.get('ivp_entry', 0.5)
    if ivp >= config['ivp_extreme_threshold']:
        p_fill *= config.get('pfill_ivp_extreme_mult', 1.0)
    elif ivp >= config['ivp_high_threshold']:
        p_fill *= config.get('pfill_ivp_high_mult', 1.0)
    
    # Clamp to valid range
    return max(config['pfill_min'], min(config['pfill_max'], p_fill))

In [20]:
def try_probabilistic_fill(p_fill, rng):
    """
    Simulate probabilistic fill by drawing uniform random number.
    
    Args:
        p_fill: Fill probability (0.0 to 1.0)
        rng: numpy random number generator
    
    Returns:
        tuple: (filled: bool, u: float) where u is the random draw
    """
    u = rng.uniform(0, 1)
    filled = (u <= p_fill)
    return filled, u



In [21]:

# Print summary of fill assumptions
print("=" * 60)
print("FILL ASSUMPTIONS BY SCENARIO")
print("=" * 60)
print(f"{'Scenario':<12} {'Entry (Sell)':<25} {'Exit (Buy Back)':<25}")
print("-" * 60)
print(f"{'Pessimistic':<12} {'Mid - 75% of spread':<25} {'Close + 75% of range':<25}")
print(f"{'Realistic':<12} {'Mid - 30% of spread':<25} {'Close + 30% of range':<25}")
print(f"{'Optimistic':<12} {'Mid (no slippage)':<25} {'Close - 25% of range':<25}")
print("=" * 60)
print(f"\nTransaction costs: ${CONFIG['commission_per_contract'] + CONFIG['sec_fee_per_contract']:.2f}/leg")
print(f"\nLiquidity Model (regime-aware):")
print(f"  Hard reject: bid < ${CONFIG['min_bid_hard']} or spread > {CONFIG['hard_max_spread_pct']*100:.0f}%")
print(f"  Base target spread: {CONFIG['base_max_spread_pct']*100:.0f}%")
print(f"  High IV ({CONFIG['ivp_high_threshold']*100:.0f}%ile): allow {CONFIG['ivp_high_max_spread_pct']*100:.0f}%")
print(f"  Extreme IV ({CONFIG['ivp_extreme_threshold']*100:.0f}%ile): allow {CONFIG['ivp_extreme_max_spread_pct']*100:.0f}%")
print(f"  Short DTE (≤{CONFIG['short_dte_threshold']}d): +{CONFIG['short_dte_extra_spread_pct']*100:.0f}% allowed")


FILL ASSUMPTIONS BY SCENARIO
Scenario     Entry (Sell)              Exit (Buy Back)          
------------------------------------------------------------
Pessimistic  Mid - 75% of spread       Close + 75% of range     
Realistic    Mid - 30% of spread       Close + 30% of range     
Optimistic   Mid (no slippage)         Close - 25% of range     

Transaction costs: $0.66/leg

Liquidity Model (regime-aware):
  Hard reject: bid < $0.1 or spread > 20%
  Base target spread: 8%
  High IV (70%ile): allow 12%
  Extreme IV (90%ile): allow 15%
  Short DTE (≤7d): +2% allowed
